In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup as BS
import pandas as pd
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#cleanup function replace tab, newline, (, )
def cleanUpString(param): 
    retValue = ""
    retValue = (param.replace('\t','')
                .replace('\n','')
                .replace('(','') 
                .replace(')','') 
                .strip()
               )
    #print ("Split")
    return(retValue)

In [4]:
#Set the first URL Link
URL = "https://www.opensecrets.org/races/candidates?cycle=2020&id=AL01&spec=N"


In [5]:
#soup.find_all("div", class_="Members--list-item")
#soup.find('div', attrs={'class' : 'Members--list-item'})

In [6]:
#Create empty dataframe
df_candidate = pd.DataFrame(columns = ['Name', 
                             'Party', 
                             'State', 
                             "DistrictNumber",
                             "Incumbment",
                             "WonRace",
                             "votePercentage",
                             "AmountRaised",
                             "AmountSpent"
                            ])

In [7]:

#function gererate_districtLinks
#TODO add State
def gererate_districtLinks(urlLink, state):
    #Create sublinks from the page for State
    districtLinks = []
    baseURL = urlLink[0: urlLink.index("/", 9)]
    page = requests.get(urlLink)
    soupX = BS(page.content, "html.parser")

    # find all districts page links in a state
    for idx, subNavLinks in enumerate(soupX.find_all("a", class_="SubNav-link")):
        res= re.findall("id=\w{2}\d+", subNavLinks["href"])
        if(res) :
            districtLinks.append(baseURL + subNavLinks["href"].replace("summary?","candidates?"))
    
    return districtLinks
#End function gererate_districtLinks

#function screen_scrape
def screen_scrape(urlLink, file_name):
    page = requests.get(urlLink)
    soup = BS(page.content, "html.parser")
    
    #df_candidate declare as global inside function to reuse this variable
    global df_candidate
    
    #Declare and initialize variable with defaults

    candidateName=""
    candidateParty=""
    state = ""
    districtNumber =""
    incumbment = ""
    wonRace = ""
    votePercentage = np.NaN
    amountRaised = np.NaN
    amountSpent = np.NaN
    
    candidateString = ""

    title = soup.find_all('title', limit=1)[0].text.split(" ")
    #state = title[0]
    state = soup.find_all('title', limit=1)[0].text.split("District")[0]
    #districtNumber = title[2]
    distSplit = soup.find_all('title', limit=1)[0].text.split("District ")[1].split(" ")    
    districtNumber = distSplit[0]

    for idx, stateMembers in enumerate(soup.find_all("div", class_="Members--list-item")):   
        bio = stateMembers.find('strong').text
        #print (bio)        
        candidateNameSplit = bio.split("(", 1) # Max 1 occurence
        candidateName = cleanUpString(str(candidateNameSplit[0])) if len(candidateNameSplit) > 1 else ""

        votePercentageSplit = bio.rsplit("(",1)    
        votePercentage = cleanUpString(str(votePercentageSplit[1])) if len(votePercentageSplit) > 1 else ""
        votePercentage= votePercentage.split("%")[0]

        incumbment = str("Incumbent") if bio.find("Incumbent") > -1 else ""
        wonRace = str("Winner") if bio.find("Winner") > -1 else ""        

        if(re.findall("\(\w\)", bio)) :        
            candidateParty = cleanUpString(re.findall("\(\w\)", bio)[0])

        amountRaised = stateMembers.find_all("td", class_="Members--number")[0].text
        amountSpent = stateMembers.find_all("td", class_="Members--number")[1].text    
        # append rows to an empty DataFrame
        df_candidate = df_candidate.append({'Name' : candidateName, 'Party' : candidateParty, 'State' : state,
                       'DistrictNumber' : districtNumber, 'Incumbment' : incumbment, 'WonRace' :wonRace,
                       'votePercentage' : votePercentage, 'AmountRaised' :amountRaised, 'AmountSpent' : amountSpent},
                    ignore_index = True)

    #df_candidate
    
#end function screen_scrape

In [8]:
#screen_scrape(URL, "file_name")
#df_candidate

In [9]:
# Create States List
#create a list with abbreviations
state_abbr = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA',
'WA','WV','WI','WY']

print(state_abbr)


['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


In [10]:
statesURL = "https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120"
page = requests.get(statesURL)
soupStates = BS(page.content, "html.parser")
#print(soupStates.prettify())
stateList = soupStates.find_all("div", class_="md-drag md-table-wrapper")[0].find_all('tr')

#Create an empty States dataframe
dfStates = pd.DataFrame(columns = ['State', 'State_Short','Representatives'])
for state in stateList:
    #print( state)
    strStateName = ""
    strReps = ""
    name = state.find_all("td", attrs={'headers' : 's'})
    reps = state.find_all("td", attrs={'headers' : 'r'})
    if( len(name) > 0) :
        strStateName = name[0].text.replace("\n","").strip()
    if( len(reps) > 0) :
        strReps = reps[0].text.replace("\n","").strip()    
    dfStates = dfStates.append({'State' : strStateName , "State_Short":len(strStateName), "Representatives":strReps},ignore_index = True)

#dfStates
#cleanup Data
dfStates = dfStates.drop([0, 51])


#Assign abbreviation to data frame
dfStates["State_Short"] = state_abbr
print(dfStates)

             State State_Short Representatives
1          Alabama          AL               7
2           Alaska          AK               1
3          Arizona          AZ               9
4         Arkansas          AR               4
5       California          CA              53
6         Colorado          CO               7
7      Connecticut          CT               5
8         Delaware          DE               1
9          Florida          FL              27
10         Georgia          GA              14
11          Hawaii          HI               2
12           Idaho          ID               2
13        Illinois          IL              18
14         Indiana          IN               9
15            Iowa          IA               4
16          Kansas          KS               4
17        Kentucky          KY               6
18       Louisiana          LA               6
19           Maine          ME               2
20        Maryland          MD               8
21   Massachu

In [11]:
# Create the links for district 1 for each state

urlLinkDistOneTemplate = "https://www.opensecrets.org/races/candidates?cycle=2020&id=STATE_ABBR01&spec=N"

USFirstStateDistLinks= []
USStateDistLinks= []
for index, row in dfStates.iterrows():
    #print (index,row["State"], row["State_Short"])
    strlnk = urlLinkDistOneTemplate.replace("STATE_ABBR",row["State_Short"] )
    USFirstStateDistLinks.append(strlnk)
    linksDistricts = gererate_districtLinks(strlnk, "STATE")
    #print(linksDistricts)
    
    for link in linksDistricts:
        ##print(link)
        USStateDistLinks.append(link)
        ##screen_scrape(link, "file_name")
print(USStateDistLinks)


['https://www.opensecrets.org/races/candidates?cycle=2020&id=AL01&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL02&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL03&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL04&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL05&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL06&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL07&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AK01&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ01&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ02&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ03&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ04&spec=N', 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ05&spec=N', 'https://www.opensecrets.org/races/ca

In [12]:
#len(USStateDistLinks)

In [13]:
for link in USStateDistLinks[0:100]:
        #print(link)        
        screen_scrape(link, "file_name")
df_candidate

,Name,Party,State,DistrictNumber,Incumbment,WonRace,votePercentage,AmountRaised,AmountSpent
0,Jerry Carl,R,Alabama,01,,Winner,64.9,"$1,971,321","$1,859,349"
1,James Averhart,D,Alabama,01,,,35.0,"$80,095","$78,973"
2,Barry Moore,R,Alabama,02,,Winner,65.3,"$650,807","$669,368"
3,Phyllis Harvey-Hall,D,Alabama,02,,,34.6,"$56,050","$55,988"
4,Mike D Rogers,R,Alabama,03,Incumbent,Winner,67.5,"$1,193,111","$1,218,564"
...,...,...,...,...,...,...,...,...,...
195,Dana Cottrell,D,Florida,11,,,33.3,"$62,658","$62,380"
196,Gus Bilirakis,R,Florida,12,Incumbent,Winner,62.9,"$1,604,137","$1,548,634"
197,Kim Walker,D,Florida,12,,,37.1,"$42,802","$33,544"
198,Charlie Crist,D,Florida,13,Incumbent,Winner,53.0,"$2,720,646","$3,980,695"


In [14]:

print(df_candidate.groupby(['State'])['State'].count())

State
Alabama          10
Alaska            2
Arizona          19
Arkansas          7
California      100
Colorado         18
Connecticut      11
Delaware          3
Florida          30
Name: State, dtype: int64
